In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import torch.nn.functional as F
import os

In [2]:
# random seed --> reproducibility
import random
random.seed(42)
torch.manual_seed(42)

In [3]:
# transforms
data_transforms = {
    'train': transforms.Compose([
    transforms.Resize((227, 227)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]),

    'test': transforms.Compose([
    transforms.Resize((227, 227)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
}

In [4]:
train_dir = '../input/chest-xray-pneumonia/chest_xray/train'
test_dir = '../input/chest-xray-pneumonia/chest_xray/test'

In [5]:
train_data = datasets.ImageFolder(root=train_dir, transform=data_transforms['train'])
test_data = datasets.ImageFolder(root=test_dir, transform=data_transforms['test'])

In [6]:
train_data

Dataset ImageFolder
    Number of datapoints: 5216
    Root location: ../input/chest-xray-pneumonia/chest_xray/train
    StandardTransform
Transform: Compose(
               Resize(size=(227, 227), interpolation=bilinear, max_size=None, antialias=warn)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [7]:
train_data[0]

(tensor([[[-0.8275, -0.8431, -0.8510,  ..., -0.2549, -0.2549, -0.2549],
          [-0.8275, -0.8353, -0.8510,  ..., -0.2627, -0.2549, -0.2549],
          [-0.8196, -0.8353, -0.8510,  ..., -0.2706, -0.2627, -0.2706],
          ...,
          [-0.7647, -0.7725, -0.7804,  ..., -0.5294, -0.5216, -0.5216],
          [-0.7255, -0.7255, -0.7333,  ..., -0.3647, -0.3569, -0.3490],
          [-0.6471, -0.6471, -0.6471,  ..., -0.1608, -0.1451, -0.1451]],
 
         [[-0.8275, -0.8431, -0.8510,  ..., -0.2549, -0.2549, -0.2549],
          [-0.8275, -0.8353, -0.8510,  ..., -0.2627, -0.2549, -0.2549],
          [-0.8196, -0.8353, -0.8510,  ..., -0.2706, -0.2627, -0.2706],
          ...,
          [-0.7647, -0.7725, -0.7804,  ..., -0.5294, -0.5216, -0.5216],
          [-0.7255, -0.7255, -0.7333,  ..., -0.3647, -0.3569, -0.3490],
          [-0.6471, -0.6471, -0.6471,  ..., -0.1608, -0.1451, -0.1451]],
 
         [[-0.8275, -0.8431, -0.8510,  ..., -0.2549, -0.2549, -0.2549],
          [-0.8275, -0.8353,

In [8]:
# DataLoader object creation for batch 
train_loader = torch.utils.data.DataLoader(train_data,
                                           batch_size = 64,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(test_data,
                                           batch_size = 64,
                                           shuffle=False)

In [9]:
# example
iterate = iter(train_loader)
sample, labels = next(iterate)

In [10]:
sample.shape

torch.Size([64, 3, 227, 227])

In [11]:
labels

tensor([0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1,
        1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1])

### AlexNet
Similar to LeNet, it has convolution layer, pooling layers and ReLU activation functions. 

New concepts:
- nn.Sequential : We provide Sequential with multiple layers and when the Sequential module is called it will apply each layer, in order, to the input. 
- nn.dropout : Dropout is a form of regularization. As our models get larger, to perform more accurately on richer datasets, they start having a significantly higher number of parameters. The problem with lots of parameters is that our models begin to overfit.

**Dropout** works by randomly setting a certain fraction, 0.5 here, of the neurons in a layer to zero. This effectively adds noise to the training of the neural network and causes neurons to learn with "less" data as they are only getting half of the information from a previous layer with dropout applied. 

In [15]:
# shape evaluation
conv1 = nn.Conv2d(3,8,11,stride=4) #first cnn
maxpool = nn.MaxPool2d(kernel_size=3, stride=2) #maxpool
conv2 = nn.Conv2d(8,12,5,padding=2) #second
conv3 = nn.Conv2d(12,16,3,padding=1) #third
conv4 = nn.Conv2d(16,20,3,padding=1) #fourth
conv5 = nn.Conv2d(20,24,3,padding=1) #fifth

#cnn layer1
x = conv1(sample)
print(x.shape)
x = maxpool(x)
print(x.shape)

#cnn layer2
x = conv2(x)
print(x.shape)
x = maxpool(x)
print(x.shape)

# cnn layer3
x = conv3(x)
print(x.shape)

# cnn layer4
x = conv4(x)
print(x.shape)

# cnn layer5
x = conv5(x)
print(x.shape)
x = maxpool(x)
print(x.shape)

torch.Size([64, 8, 55, 55])
torch.Size([64, 8, 27, 27])
torch.Size([64, 12, 27, 27])
torch.Size([64, 12, 13, 13])
torch.Size([64, 16, 13, 13])
torch.Size([64, 20, 13, 13])
torch.Size([64, 24, 13, 13])
torch.Size([64, 24, 6, 6])


In [18]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet,self).__init__()
        
        self.features = torch.nn.Sequential(
            # conv layer 1
            nn.Conv2d(in_channels=3,
                      out_channels=8,
                      kernel_size=11,
                      stride=4), 
            nn.MaxPool2d(kernel_size=3,
                        stride=2),  
            nn.ReLU(inplace=True),
            
            # conv layer 2
            nn.Conv2d(in_channels=8,
                      out_channels=12,
                      kernel_size=5,
                      padding=2),#stride=1(default)
            nn.MaxPool2d(kernel_size=3,
                        stride=2),
            nn.ReLU(inplace=True),
            
            # conv layer 3
            nn.Conv2d(in_channels=12,
                      out_channels=16,
                      kernel_size=3,
                      padding=1), #stride=1(default)
            nn.ReLU(inplace=True),
            
            # conv layer 4
            nn.Conv2d(in_channels=16,
                      out_channels=20,
                      kernel_size=3,
                      padding=1), #stride=1(default)
            nn.ReLU(inplace=True),
            
            # conv layer 5
            nn.Conv2d(in_channels=20,
                      out_channels=24,
                      kernel_size=3,
                      padding=1), #stride=1(default)
            nn.MaxPool2d(kernel_size=3,
                        stride=2),
            nn.ReLU(inplace=True))
                
        self.classifier = torch.nn.Sequential(
            #linear layer 1
            nn.Dropout(0.5),
            nn.Linear(24*6*6, 512),
            nn.ReLU(inplace=True),
        
            #linear layer 2
            nn.Dropout(0.5),
            nn.Linear(512,512),
            nn.ReLU(inplace=True),
        
            #linear layer 3
            nn.Linear(512,2))
        
    def forward(self,x):
        # convolution
        x = self.features(x)
        
        #flatten
        x = x.view(-1, 24*6*6)
        
        # fully connected layer
        x = self.classifier(x)
        
        return x

In [19]:
# class object
model = AlexNet()

In [20]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 720,874 trainable parameters


In [21]:
# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [22]:
# total iteration
len(train_loader)

82

In [23]:
total_batches = len(train_loader)

for epoch in range(10):
    for i , (images, labels) in enumerate(train_loader):
        
        # forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 40 == 0:
            print(f'Epoch [{epoch+1}/{10}], step [{i+1}/{total_batches}], loss:{loss.item():.4f}')
            
print('Finished Training')    

Epoch [1/10], step [40/82], loss:0.4127
Epoch [1/10], step [80/82], loss:0.3114
Epoch [2/10], step [40/82], loss:0.1189
Epoch [2/10], step [80/82], loss:0.1442
Epoch [3/10], step [40/82], loss:0.1492
Epoch [3/10], step [80/82], loss:0.0521
Epoch [4/10], step [40/82], loss:0.1801
Epoch [4/10], step [80/82], loss:0.1319
Epoch [5/10], step [40/82], loss:0.0779
Epoch [5/10], step [80/82], loss:0.1193
Epoch [6/10], step [40/82], loss:0.0885
Epoch [6/10], step [80/82], loss:0.0876
Epoch [7/10], step [40/82], loss:0.1407
Epoch [7/10], step [80/82], loss:0.1067
Epoch [8/10], step [40/82], loss:0.0350
Epoch [8/10], step [80/82], loss:0.0507
Epoch [9/10], step [40/82], loss:0.0906
Epoch [9/10], step [80/82], loss:0.0603
Epoch [10/10], step [40/82], loss:0.1477
Epoch [10/10], step [80/82], loss:0.1158
Finished Training


In [24]:
# save this model
torch.save(model.state_dict(), 'modelAlexNet.pth')

In [25]:
# evaluation and testing 
with torch.no_grad():
    n_correct =0
    n_samples =0
    
    for images, labels in test_loader:
        # forward -- softmax prediction 
        outputs = model(images)
        
        # actual prediction
        # value, index
        _, predictions = torch.max(outputs, 1) # multiclass pred
        n_samples += labels.shape[0]
        n_correct += (predictions==labels).sum().item()
        
    acc = 100.0 * n_correct / n_samples
    print(acc)

72.59615384615384


## Initializing parameters in AlexNet

In [26]:
## with initializing parameters
def initialize_parameters(m):
    if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight.data, nonlinearity='relu')
        nn.init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight.data, gain=nn.init.calculate_gain('relu'))
        nn.init.constant_(m.bias.data, 0)

In [27]:
model.apply(initialize_parameters)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 8, kernel_size=(11, 11), stride=(4, 4))
    (1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU(inplace=True)
    (3): Conv2d(8, 12, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): ReLU(inplace=True)
    (6): Conv2d(12, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(16, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(20, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): ReLU(inplace=True)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=864, out_features=512, bias=True)
    (2): ReLU(inplace=True)
    (3): Dropout(p=0.5, inplace=False)
    (4): Lin

In [28]:
total_batches = len(train_loader)

for epoch in range(10):
    for i , (images, labels) in enumerate(train_loader):
        
        # forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 40 == 0:
            print(f'Epoch [{epoch+1}/{10}], step [{i+1}/{total_batches}], loss:{loss.item():.4f}')
            
print('Finished Training')    

Epoch [1/10], step [40/82], loss:0.4085
Epoch [1/10], step [80/82], loss:0.1429
Epoch [2/10], step [40/82], loss:0.1487
Epoch [2/10], step [80/82], loss:0.2438
Epoch [3/10], step [40/82], loss:0.2408
Epoch [3/10], step [80/82], loss:0.1726
Epoch [4/10], step [40/82], loss:0.3096
Epoch [4/10], step [80/82], loss:0.1513
Epoch [5/10], step [40/82], loss:0.1123
Epoch [5/10], step [80/82], loss:0.0681
Epoch [6/10], step [40/82], loss:0.1082
Epoch [6/10], step [80/82], loss:0.1421
Epoch [7/10], step [40/82], loss:0.0957
Epoch [7/10], step [80/82], loss:0.1061
Epoch [8/10], step [40/82], loss:0.0848
Epoch [8/10], step [80/82], loss:0.1623
Epoch [9/10], step [40/82], loss:0.0835
Epoch [9/10], step [80/82], loss:0.1984
Epoch [10/10], step [40/82], loss:0.1573
Epoch [10/10], step [80/82], loss:0.1095
Finished Training


In [29]:
# save this model
torch.save(model.state_dict(), 'modelAlexNet_initialparam.pth')

In [30]:
# evaluation and testing 
with torch.no_grad():
    n_correct =0
    n_samples =0
    
    for images, labels in test_loader:
        # forward -- softmax prediction 
        outputs = model(images)
        
        # actual prediction
        # value, index
        _, predictions = torch.max(outputs, 1) # multiclass pred
        n_samples += labels.shape[0]
        n_correct += (predictions==labels).sum().item()
        
    acc = 100.0 * n_correct / n_samples
    print(acc)

75.48076923076923
